In [1]:
# pip install --upgrade transformers


In [2]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, pipeline

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load dataset
train_data = pd.read_csv('../Dataset/train_preprocessed.csv')
print(f"Dataset shape: {train_data.shape}")
print(f"Sample data:\n{train_data.head(2)}")

# Check if any required columns are missing
required_columns = ['comment_text'] + ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
missing_columns = [col for col in required_columns if col not in train_data.columns]
if missing_columns:
    raise ValueError(f"Missing columns in dataset: {missing_columns}")

# Format data
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
def format_labels(row):
    return ', '.join([f"{col}={int(row[col])}" for col in label_cols])  # Ensure integers

# Handle potential NaNs
train_data['comment_text'] = train_data['comment_text'].fillna("")
for col in label_cols:
    train_data[col] = train_data[col].fillna(0).astype(int)

# Create formatted text
train_data['input'] = train_data.apply(format_labels, axis=1)
train_data['output'] = train_data['comment_text']
train_data['text'] = "<toxicity> " + train_data['input'] + " </toxicity> <comment> " + train_data['output']

# Convert to dataset
dataset = Dataset.from_pandas(train_data[['text']])
print(f"Created dataset with {len(dataset)} examples")

# Tokenize with the right format for causal language modeling
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Tokenize inputs
    result = tokenizer(
        examples['text'],
        truncation=True,
        padding="max_length",
        max_length=256  # Increased max length
    )
    
    # Set up labels for language modeling (same as input_ids)
    result["labels"] = result["input_ids"].copy()
    return result

# Tokenize dataset
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=['text']  # Remove text column as it's not needed after tokenization
)
print(f"Tokenized dataset: {tokenized_dataset}")

# Configure training
training_args = TrainingArguments(
    output_dir="./gpt2-toxic",
    per_device_train_batch_size=3,  # Reduced batch size
    gradient_accumulation_steps=4,  # Add gradient accumulation
    num_train_epochs=2,
    logging_steps=10,
    max_steps=100,
    save_steps=500,
    save_total_limit=2,
    learning_rate=5e-5,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # Only use fp16 if GPU is available
)

# Load model and configure trainer
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Resize for any special tokens

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

# Train
try:
    trainer.train()
    # Save model
    model.save_pretrained("./gpt2-toxic-final")
    tokenizer.save_pretrained("./gpt2-toxic-final")
    print("Training completed successfully!")
except Exception as e:
    print(f"Training failed with error: {e}")

2025-06-03 11:27:23.457305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748942843.503620   23195 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748942843.519404   23195 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748942843.619380   23195 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748942843.619402   23195 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748942843.619404   23195 computation_placer.cc:177] computation placer alr

Using device: cuda
Dataset shape: (159571, 12)
Sample data:
                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   

   severe_toxic  obscene  threat  insult  identity_hate  \
0             0        0       0       0              0   
1             0        0       0       0              0   

                                      processed_text  original_length  \
0  explanation edits made my username hardcore me...              264   
1  daww ! he match background colour im seemingly...              112   

   processed_length  length_reduction  
0               202         23.484848  
1                86         23.214286  
Created dataset with 159571 examples


Map:   0%|          | 0/159571 [00:00<?, ? examples/s]

Tokenized dataset: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 159571
})


/tmp/ipykernel_23195/2192023697.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,6.856500
20,6.480700
30,5.687500
40,4.119500
50,2.508500
60,1.887900
70,1.600100
80,1.442600
90,1.244400
100,1.470800


Training completed successfully!


In [3]:
# Reload model if needed
# from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
# model = GPT2LMHeadModel.from_pretrained("./gpt2-toxic")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example prompt
prompt = "<toxicity> toxic=1, obscene=1, insult=1 </toxicity> <comment>"
output = text_generator(prompt, max_length=50, do_sample=True, top_k=50, top_p=0.95)
print(output[0]['generated_text'])


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<toxicity> toxic=1, obscene=1, insult=1 </toxicity> <comment> It is also important to note that the first sentence of the above sentence is not intended to mean that the sentence is not a reference to the current definition of "toxic". The only reference to that definition is to the fact that the definition of "toxic" is based on a definition that is very clearly defined and is very similar to the definition of "toxic" and "toxic" in the law. It is important to note that the reference to "toxic" was in the original law, as well as the new definition of "toxic" and "toxic" was inserted to make it more clear that it is not a reference to the current definition. This does not mean that "toxic" is not a reference to any specific crime, but that is simply a reference to a specific crime. The question is whether the statute is used in the context of an "attempt to harm" or "attempt to injure". The statute does not directly refer to any specific crime or to any specific crime. The statute doe

In [4]:
prompt = "<toxicity>  obscene=1 </toxicity> <comment>"
output = text_generator(prompt, max_length=25, do_sample=True, top_k=50, top_p=0.95)
print(output[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=25) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<toxicity>  obscene=1 </toxicity> <comment> No, I don't think you're saying you don't want to kill?


In [5]:
# Reload model if needed
# from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer
# model = GPT2LMHeadModel.from_pretrained("./gpt2-toxic")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

text_generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Example prompt
prompt = "<toxicity> toxic=1, obscene=1, insult=1 </toxicity> <comment>"
output = text_generator(prompt, max_length=50, do_sample=True, top_k=50, top_p=0.95)
print(output[0]['generated_text'])


Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<toxicity> toxic=1, obscene=1, insult=1 </toxicity> <comment> "As you can see, this is the worst case for what I would recommend to my friend, I was just a bit too young for this. And my friend is a teenager and I am not sure if I understand what you are saying. But I believe it is worth considering as a possibility. My friend's age, I have never experienced a person I was so young for in this world, and I am aware that it is a very dangerous place to be. That is why I have decided to ask you to take a look at the story of my friend's death. I will be taking it very seriously. I am sure that there is some truth in it. I will take you to the death of my friend. I am sure that the last part of it will be written by the last person who will be alive, that is because of the same person who died in the last few days. The last person who will be alive will be my friend and I am sure that it is not possible to get the information out of this case. I am going to be very careful to tell you wha

In [12]:
# Example prompt
prompt = "<toxicity> toxic=1, obscene=1, insult=1 </toxicity> <comment>"
output = text_generator(prompt, max_length=56, do_sample=True, top_k=50, top_p=0.95)
print(output[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=56) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<toxicity> toxic=1, obscene=1, insult=1 </toxicity> <comment> The way to describe this is to say "You know how I feel? I think I'm not the only one who feels this way. I'd rather have a lot of people that are on the same side as you, but I think it's better to have a lot of people that you can be friends with, but that's it. It's not that different than having someone who is not on the same side as you. You know that, you know that too, you know that. I think that's a good thing. I think I would like to see people who are on the same side as you, but I think we should be able to come together, but I don't want that to be the case. You know, we should be able to do things together. I want to see people who are not on the same side as you, but we should be able to be friends. We should be able to do things together. If we are not able to do that, I think we should be able to do things."
